In [12]:
from ultralytics import YOLO
import cv2

# Load the YOLO model
model = YOLO(r"C:\Users\HP\Downloads\yolov5nu.pt")

# Track and process the video
vid = cv2.VideoCapture(r"C:\Users\HP\Downloads\YOLOv8_Realtime_Car_Detection_Tracking_and_counting-master\YOLOv8_Realtime_Car_Detection_Tracking_and_counting-master\test3.mp4")

# Global variables for tracking
trackers = []
tracked_objects = {}
object_id_counter = 0

def start_tracking(frame, bbox, class_name):
    global trackers, object_id_counter, tracked_objects
    tracker = cv2.TrackerCSRT_create()
    tracker.init(frame, bbox)
    object_id_counter += 1
    tracked_objects[object_id_counter] = {
        "class_name": class_name,
        "tracker": tracker,
        "path": []
    }
    return object_id_counter

while True:
    ret, frame = vid.read()
    if not ret:
        break

    # Update existing trackers
    for object_id, obj_data in list(tracked_objects.items()):
        tracker = obj_data["tracker"]
        success, bbox = tracker.update(frame)
        if success:
            x, y, w, h = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            class_name = obj_data["class_name"]
            cv2.putText(frame, f"{class_name} {object_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)

            # Update the car path
            obj_data["path"].append((x + w // 2, y + h // 2))
            for i in range(1, len(obj_data["path"])):
                if i % 3 == 0:
                    cv2.line(frame, obj_data["path"][i - 1], obj_data["path"][i], (0, 0, 255), 2)
        else:
            del tracked_objects[object_id]

    # Detect new cars
    results = model.predict(frame)
    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        conf = result.conf[0]
        cls = int(result.cls[0])
        label = f"{model.names[cls]} {conf:.2f}"

        if model.names[cls] == 'car':
            bbox = (x1, y1, x2 - x1, y2 - y1)
            object_id = start_tracking(frame, bbox, model.names[cls])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{model.names[cls]} {object_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)

    cv2.imshow('Detected Cars', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()



0: 384x640 1 car, 204.5ms
Speed: 0.0ms preprocess, 204.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 109.8ms
Speed: 0.0ms preprocess, 109.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 109.8ms
Speed: 0.0ms preprocess, 109.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 94.1ms
Speed: 0.0ms preprocess, 94.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 94.2ms
Speed: 0.0ms preprocess, 94.2ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 78.6ms
Speed: 15.6ms preprocess, 78.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 78.6ms
Speed: 0.0ms preprocess, 78.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 78.6ms
Speed: 15.6ms preprocess, 78.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

In [12]:
from ultralytics import YOLO
import cv2

# Load the YOLO model
model = YOLO(r"C:\Users\HP\Downloads\yolov5nu.pt")

# Track and process the webcam feed
cap = cv2.VideoCapture(0)

# Global variables for tracking
trackers = []
tracked_objects = {}
object_id_counter = 0

def start_tracking(frame, bbox, class_name):
    global trackers, object_id_counter, tracked_objects
    tracker = cv2.TrackerCSRT_create()
    tracker.init(frame, bbox)
    object_id_counter += 1
    tracked_objects[object_id_counter] = {
        "class_name": class_name,
        "tracker": tracker,
        "path": []
    }
    return object_id_counter

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Update existing trackers
    for object_id, obj_data in list(tracked_objects.items()):
        tracker = obj_data["tracker"]
        success, bbox = tracker.update(frame)
        if success:
            x, y, w, h = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            class_name = obj_data["class_name"]
            cv2.putText(frame, f"{class_name} {object_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)

            # Update the car path
            obj_data["path"].append((x + w // 2, y + h // 2))
            for i in range(1, len(obj_data["path"])):
                if i % 3 == 0:
                    cv2.line(frame, obj_data["path"][i - 1], obj_data["path"][i], (0, 0, 255), 2)
        else:
            del tracked_objects[object_id]

    # Detect new cars
    results = model.predict(frame)
    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        conf = result.conf[0]
        cls = int(result.cls[0])
        label = f"{model.names[cls]} {conf:.2f}"

        if model.names[cls] == 'car' and conf > 0.5:
            bbox = (x1, y1, x2 - x1, y2 - y1)
            object_id = None
            for i, obj_data in tracked_objects.items():
                tracker = obj_data["tracker"]
                success, obj_bbox = tracker.update(frame)
                if success:
                    obj_x, obj_y, obj_w, obj_h = [int(v) for v in obj_bbox]
                    if x1 >= obj_x and y1 >= obj_y and x2 <= obj_x + obj_w and y2 <= obj_y + obj_h:
                        object_id = i
                        break
            if object_id is None:
                object_id = start_tracking(frame, bbox, model.names[cls])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{model.names[cls]} {object_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)

    cv2.imshow('Webcam Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 2 cars, 1 traffic light, 616.0ms
Speed: 3.0ms preprocess, 616.0ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 1 traffic light, 294.7ms
Speed: 4.0ms preprocess, 294.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 1 traffic light, 242.1ms
Speed: 4.5ms preprocess, 242.1ms inference, 4.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cars, 1 traffic light, 187.7ms
Speed: 4.0ms preprocess, 187.7ms inference, 12.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 cars, 1 traffic light, 201.3ms
Speed: 3.5ms preprocess, 201.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cars, 160.1ms
Speed: 7.0ms preprocess, 160.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 1 traffic light, 176.5ms
Speed: 4.5ms preprocess, 176.5ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car,

In [10]:
from ultralytics import YOLO
import cv2

# Load the YOLO model
model = YOLO(r"C:\Users\HP\Downloads\yolov5nu.pt")

# Track and process the webcam feed
cap = cv2.VideoCapture(0)

# Global variables for tracking
trackers = []
tracked_objects = {}
object_id_counter = 0

def start_tracking(frame, bbox, class_name):
    global trackers, object_id_counter, tracked_objects
    tracker = cv2.TrackerCSRT_create()
    tracker.init(frame, bbox)
    object_id_counter += 1
    tracked_objects[object_id_counter] = {
        "class_name": class_name,
        "tracker": tracker,
        "path": []
    }
    return object_id_counter

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Update existing trackers
    for object_id, obj_data in list(tracked_objects.items()):
        tracker = obj_data["tracker"]
        success, bbox = tracker.update(frame)
        if success:
            x, y, w, h = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            class_name = obj_data["class_name"]
            cv2.putText(frame, f"{class_name} {object_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)

            # Update the car path
            obj_data["path"].append((x + w // 2, y + h // 2))
            for i in range(1, len(obj_data["path"])):
                cv2.line(frame, obj_data["path"][i - 1], obj_data["path"][i], (0, 0, 255), 2)
        else:
            del tracked_objects[object_id]

    # Detect new cars
    results = model.predict(frame)
    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        conf = result.conf[0]
        cls = int(result.cls[0])
        label = f"{model.names[cls]} {conf:.2f}"

        if model.names[cls] == 'car' and conf > 0.5:
            bbox = (x1, y1, x2 - x1, y2 - y1)
            object_id = None
            for i, obj_data in tracked_objects.items():
                tracker = obj_data["tracker"]
                success, obj_bbox = tracker.update(frame)
                if success:
                    obj_x, obj_y, obj_w, obj_h = [int(v) for v in obj_bbox]
                    if x1 >= obj_x and y1 >= obj_y and x2 <= obj_x + obj_w and y2 <= obj_y + obj_h:
                        object_id = i
                        break
            if object_id is None:
                object_id = start_tracking(frame, bbox, model.names[cls])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{model.names[cls]} {object_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)

    cv2.imshow('Webcam Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 771.7ms
Speed: 6.2ms preprocess, 771.7ms inference, 15.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 330.7ms
Speed: 10.7ms preprocess, 330.7ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 224.0ms
Speed: 4.0ms preprocess, 224.0ms inference, 5.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 209.7ms
Speed: 4.8ms preprocess, 209.7ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 1 laptop, 211.5ms
Speed: 3.5ms preprocess, 211.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
